In [ ]:
import numpy as np

In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.initializers import Constant
from tensorflow.keras.initializers import Initializer
from tensorflow.keras.layers import Dense, Flatten

In [ ]:
# Load MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Preprocess data
X_train = X_train.reshape(-1, 784).astype('float32') / 255
X_test = X_test.reshape(-1, 784).astype('float32') / 255
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [ ]:
# Custom initializers
class CustomKernelInitializer(Initializer):
    def __init__(self, weights):
        self.weights = weights

    def __call__(self, shape, dtype=None):
        return self.weights

class CustomBiasInitializer(Initializer):
    def __init__(self, biases):
        self.biases = biases

    def __call__(self, shape, dtype=None):
        return self.biases

In [ ]:
energy_threshold = 0.9

In [ ]:
# Function to count the node connections in the model
def count_node_connections(model):
    total_connections = 0
    for layer in model.layers:
        if isinstance(layer, Dense):
            weights = layer.get_weights()
            connections = 0
            for weight in weights: # Iterate over weights and biases
                connections += weight.size  # Calculate connections for each array
            print(f'Layer: {layer.name}, Connections: {connections}')
            total_connections += connections
    return total_connections

In [ ]:
# Define the original model
original_model = Sequential([
    Dense(128, input_shape=(784,), activation='relu'),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

original_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the original model
original_model.fit(X_train, y_train, epochs=5, batch_size=128, validation_data=(X_test, y_test))

# Evaluate the original model
loss, accuracy = original_model.evaluate(X_test, y_test)
print(f'Original Model Accuracy: {accuracy * 100:.2f}%')

original_model.summary()

# Count the node connections in the model
total_connections = count_node_connections(original_model)
print(f'Total node connections in the model: {total_connections}')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8288 - loss: 0.6061 - val_accuracy: 0.9469 - val_loss: 0.1741
Epoch 2/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9572 - loss: 0.1437 - val_accuracy: 0.9613 - val_loss: 0.1235
Epoch 3/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9712 - loss: 0.0946 - val_accuracy: 0.9691 - val_loss: 0.0956
Epoch 4/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9795 - loss: 0.0704 - val_accuracy: 0.9726 - val_loss: 0.0858
Epoch 5/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9830 - loss: 0.0557 - val_accuracy: 0.9734 - val_loss: 0.0845
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9697 - loss: 0.0956
Original Model Accuracy: 97.34%


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 128)                 │         100,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 10)                  │             650 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 328,160 (1.25 MB)

 Trainable params: 109,386 (427.29 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 218,774 (854.59 KB)

Layer: dense, Connections: 100480
Layer: dense_1, Connections: 8256
Layer: dense_2, Connections: 650
Total node connections in the model: 109386


In [ ]:
def apply_svd(weights, energy_threshold=0.2):
    # Step 1: Compute the SVD of the matrix
    U, s, Vt = np.linalg.svd(weights, full_matrices=False)

    # Step 2: Calculate the total energy (sum of squares of all singular values)
    total_energy = np.sum(s**2)

    # Step 3: Determine the threshold for retaining singular values (20% of the total energy)
    energy_target = energy_threshold * total_energy

    # Step 4: Find the number of singular values that together account for this 20% energy
    cumulative_energy = 0
    rank = 0
    for singular_value in s:
        cumulative_energy += singular_value**2
        rank += 1
        if cumulative_energy >= energy_target:
            break

    U_reduced = U[:, :rank]
    s_reduced = np.diag(s[:rank])
    Vt_reduced = Vt[:rank, :]
    new_weights_1 = np.dot(U_reduced, s_reduced)
    new_weights_2 = Vt_reduced

    return new_weights_1, new_weights_2, rank

In [ ]:
def prune_weights(model, energy_threshold=0.2):

    #layers_to_prune = model.layers[:-1]
    layers_to_prune = model.layers

    # Create a new model with pruned layers
    new_model = Sequential()
    new_model.add(Flatten(input_shape=(784,)))

    for i, layer in enumerate(layers_to_prune):
        if isinstance(layer, Dense):
            weights, biases = layer.get_weights()

            print(i)

            new_weights_1, new_weights_2, rank = apply_svd(weights, energy_threshold)

            if np.isnan(new_weights_1).any():
              print(f"NaNs found in layer {layer.name}, weight matrix {i}")
            if np.isnan(new_weights_2).any():
              print(f"NaNs found in layer {layer.name}, weight matrix {i}")

            kernel_initializer = CustomKernelInitializer(new_weights_1)
            new_model.add(Dense(new_weights_1.shape[1], kernel_initializer = kernel_initializer, use_bias = False, activation = 'relu'))
            kernel_initializer = CustomKernelInitializer(new_weights_2)
            bias_initializer   = CustomBiasInitializer(biases)
            if i < 2:
              new_model.add(Dense(new_weights_2.shape[1], kernel_initializer = kernel_initializer, bias_initializer = bias_initializer, activation = 'relu'))
            else:
              new_model.add(Dense(new_weights_2.shape[1], kernel_initializer = kernel_initializer, bias_initializer = bias_initializer, activation = 'softmax'))

    return new_model

In [ ]:
restructured_model = prune_weights(original_model, energy_threshold)

restructured_model.summary()

# Count the node connections in the model
total_connections = count_node_connections(restructured_model)
print(f'Total node connections in the model: {total_connections}')

0
1
2


/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten (Flatten)                    │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 87)                  │          68,208 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 128)                 │          11,264 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 38)                  │           4,864 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 64)                  │           2,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 9)                   │             576 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 10)                  │             100 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 87,508 (341.83 KB)

 Trainable params: 87,508 (341.83 KB)

 Non-trainable params: 0 (0.00 B)

Layer: dense_3, Connections: 68208
Layer: dense_4, Connections: 11264
Layer: dense_5, Connections: 4864
Layer: dense_6, Connections: 2496
Layer: dense_7, Connections: 576
Layer: dense_8, Connections: 100
Total node connections in the model: 87508


In [ ]:
# Compile the restructured model
restructured_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Retrain the restructured model
restructured_model.fit(X_train, y_train, epochs=5, batch_size=128, validation_data=(X_test, y_test))

# Evaluate the restructured model
loss, accuracy = restructured_model.evaluate(X_test, y_test)
print(f'Restructured Model Accuracy: {accuracy * 100:.2f}%')

Epoch 1/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.7758 - loss: 0.6350 - val_accuracy: 0.9580 - val_loss: 0.1345
Epoch 2/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9727 - loss: 0.0931 - val_accuracy: 0.9674 - val_loss: 0.1092
Epoch 3/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9802 - loss: 0.0649 - val_accuracy: 0.9670 - val_loss: 0.1094
Epoch 4/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9822 - loss: 0.0553 - val_accuracy: 0.9735 - val_loss: 0.0935
Epoch 5/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9864 - loss: 0.0428 - val_accuracy: 0.9705 - val_loss: 0.1013
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9670 - loss: 0.1147
Restructured Model Accuracy: 97.05%
